In [1]:
import pandas as pd
from neo4j import GraphDatabase

pd.set_option('display.max_columns',None)

df = pd.read_csv('./data/restaurant_reviews_withsentiment.csv')
df_loc = pd.read_csv('./data/restaurants_latest.csv')

In [2]:
df.head(100)

,Title,Body,Reviewer,Reviewer_Level,Reviewer_NumReviews,ReviewDateTime,Restaurant,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(vader),RestaurantScore(vader),FoodFeature(vader),FoodScore(vader),SentimentClass
0,Glad It’s Back!,We ordered two appetisers and four main dishes...,Sofina Ng,3.0,9.0,10m ago,/babasan-by-uncle-kiisu,"beer,fruit",we ordered two appetisers and four main dishes...,kronenbourg blanc beer was very enjoyable too....,"main,unique,boasting,manifested,creative,strong","enjoyable,sweet,mild","creative,😍,stout,strong,definitely,🤗",0.7263,"enjoyable,sweet",0.0,1
1,Yuzu Ochazuke ($24),This rice in soup dish is the first one I’ve h...,Jan L,3.0,10.0,5d ago,/babasan-by-uncle-kiisu,rice,boy did it impress! perfect harmonious blend o...,this rice in soup dish is the first one i’ve h...,"impress,perfect,harmonious,fresh,generous,serv...","great,good","impress,perfect,harmonious,fresh,generous,reco...",0.0000,"great,good",0.0,1
2,Belly Roll Mee ($15),"The successor to Uncle Kiisu, Babasan’s menu d...",Dex Neo,8.0,958.0,Oct 24 at 5:42pm,/babasan-by-uncle-kiisu,"crispy pork,mee siam,pork,pork belly","the successor to uncle kiisu, babasan’s menu d...",i was seriously wondering how they would pull ...,"dreamt,good,sounds,heavy,provided,cut,amazingl...","seriously wondering,pull,crispy,stuffed,tender...","successor,definitely,good,cut,richness,afforda...",0.6219,"seriously,hugging,super",0.0,1
3,Samurai Kueh Pie Tee,This was like a little bomb that exploded with...,Komal Salve,7.0,208.0,Oct 23 at 10:06am,/babasan-by-uncle-kiisu,"fish,fish roe,rice,roe,seafood,unagi",this was like a little bomb that exploded with...,i'm not a big fan of the seafoody taste loved ...,"little,exploded,eating,try,super worth","big,loved,flying,smooth","like,bomb,definitely,super,worth",0.0000,"fan,loved",0.0,1
4,Build Your Own Bowl (Small),Most salad/grain bowls serve up similar stuff ...,Nom Nom,3.0,15.0,Oct 25 at 10:13pm,/yobo,"brown rice,cream,pumpkin,rice,salad",but i really enjoyed the broccoli puree served...,most salad/grain bowls serve up similar stuff ...,"enjoyed,served","salad,serve,similar,creamy,paired,brown,roaste...",enjoyed,0.0000,"pretty,good",0.0,1
5,Healthy Warm Salad Bowls At Affordable Pricing,"I ordered 2 servings of honey soy, 1 serving o...",Ray L,2.0,1.0,Oct 18 at 3:40pm,/yobo,"aglio olio,brussel sprout,fusilli,mushroom,mus...",this was all done using their online portal an...,"i ordered 2 servings of honey soy, 1 serving o...","online,paid,nice,late,eat,definitely recommend","ordered,serving,liked,nutty,strong,garlicky,st...","nice,definitely,recommend",0.0000,"delight,liked,strong,warm",0.0,1
6,Excellent Value For Money In CBD,Away with standardised salad/grain bowls in CB...,shirley L,2.0,1.0,Oct 18 at 12:47pm,/yobo,"nut,salad",you can have a bowl of 400 kcal (recommended k...,away with standardised salad/grain bowls in cb...,"kcal,recommended,including,recommend","away standardised,salad,offers,pay,add,excelle...","recommended,recommend",0.0000,"pay,bonus,excellent",0.0,1
7,For Delicious Build-Your-Own-Bowls in the CBD,Make note of this spot on Cecil Street for qui...,Burpple Guides,NaN,NaN,Oct 16 at 4:16pm,/yobo,"brown rice,chicken,cream,kale chips,rice",make note of this spot on cecil street for qui...,keep it healthy with japanese brown rice as yo...,"quick,customisable,small,includes,topping","healthy,japanese,brown,recommends,adding,cream...",NaN,0.0000,"healthy,recommends,bitterness",0.0,1
8,Pay Per Portion,Fully customizable bowl e.g any combination of...,Zac Lee,2.0,1.0,Oct 15 at 8:38am,/yobo,salad,fully customizable bowl e.g any combination of...,competitive pricing compared to other grain/sa...,"fully customizable,pay,able,select,exact,excel...","competitive,compared,salad","pay,excellent,recommend,amazing",0.0000,competitive,0.0,1
9,YOBO Grain Bowl,Newly opened grain bowl place located on the 2...,Gan Zhang Xun,6.0,179.0,Oct 8 at 1:51pm,/yobo,"chicken,cream,k

In [3]:
df_loc.head()

,Unnamed: 0,Title,Link,Location,Latitude,Longitude,NumReviews,PriceRange,Tags,Categories,Address
0,0,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Clarke Quay,1.286034,103.849552,4,~$30/pax,['BEYOND'],"['Local Delights', ' Japanese', ' 1-for-1 Deal...",50 Circular Road Singapore 049405
1,1,Enjoy Eating House and Bar,/enjoy-eating-house-and-bar,Farrer Park,1.313191,103.859697,131,~$15/pax,"['BEYOND', 'Featured In 3 Guides']","['Bars', ' Local Delights', ' Zi Char', ' Thai...",383 Jalan Besar Kam Leng Hotel Singapore 209001
2,2,Wu Pao Chun,/wu-pao-chun-bakery,City Hall,1.292939,103.851480,79,~$13/pax,"['BEYOND', 'Featured In 1 Guide']","['Bread', ' 1-for-1 Deals', ' Burpple Beyond']",13 Stamford Road #01-19/20 Capitol Singapore ...
3,3,The Salted Plum (Circular Road),/thesaltedplumsg,Raffles Place,1.287783,103.848887,215,~$20/pax,"['BEYOND', 'Hot 100 2019']","['Taiwanese', ' Cheap & Good', ' 1-for-1 Deals...",10 Circular Road Singapore 049366
4,4,Rakki Bowl,/rakki-bowl,Tanjong Pagar,1.278173,103.843013,293,~$25/pax,"['BEYOND', 'Hot 100 2019', 'Featured In 2 Guid...","['Chirashi', ' Japanese', ' 1-for-1 Deals']",72 Duxton Road Singapore 089531


In [4]:
df_loc.rename(columns={'Link':'Restaurant'},inplace=True)
df = df.merge(df_loc[['Restaurant','Address']], how='left', on='Restaurant')

In [5]:
df['Review'] = df['Title'] + ":" + df['Body']

In [6]:
df['Food'].fillna("",inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63921 entries, 0 to 63920
Data columns (total 19 columns):
Title                       63921 non-null object
Body                        58408 non-null object
Reviewer                    63921 non-null object
Reviewer_Level              63096 non-null float64
Reviewer_NumReviews         63096 non-null float64
ReviewDateTime              63921 non-null object
Restaurant                  63921 non-null object
Food                        63921 non-null object
RestaurantReview            49886 non-null object
FoodReview                  49326 non-null object
RestaurantFeature(spacy)    43195 non-null object
FoodFeature(spacy)          47828 non-null object
RestaurantFeature(vader)    37742 non-null object
RestaurantScore(vader)      63921 non-null float64
FoodFeature(vader)          40627 non-null object
FoodScore(vader)            63921 non-null float64
SentimentClass              63921 non-null int64
Address                     63921 non-

In [47]:
from py2neo import Graph, Node, Relationship
# Input the user name and password set in the neo4j graph
graph = Graph("bolt://localhost:7687", auth=("neo4j","1234"))
graph.delete_all()

In [48]:
for index, row in df[:100].iterrows():
    # Create nodes from data set
    review = Node('Review', 
                  review = row['Title']) 

    reviewer = Node('Reviewer', 
                    name = row['Reviewer'],
                    reviewer_level = row['Reviewer_Level'], 
                    no_of_reviews= row['Reviewer_NumReviews'])
    
    foodreview = Node('Food Review', 
                      food_review = row['FoodReview'], 
                      food_sentiment = row['SentimentClass'],
                      features = row['FoodFeature(spacy)'])
    
    restaurantreview = Node('Restaurant Review', 
                            restaurant_review = row['RestaurantReview'], 
                            overall_sentiment = row['SentimentClass'], 
                            features = row['RestaurantFeature(spacy)'])
    
    restaurant = Node('Restaurant', 
                      name = row['Restaurant'],
                      address= row['Address'])
    
    temp_food = row['Food'].split()
    for food in temp_food:
        if "," in food:
            food = food.split(",")
            for food in food:                
                food_node = Node('Food', 
                            name = food, 
                            features = row['FoodFeature(spacy)'])
    
    # Prevent duplicated nodes
    graph.merge(review,'Review','review')
    graph.merge(restaurant,'Restaurant','name')
    graph.merge(food_node,'Food','name')
    graph.merge(reviewer,'Reviewer','name')
    
    # Create unique relationships
    reviewer_wrote_review = Relationship(reviewer, "Wrote", review)
    graph.merge(reviewer_wrote_review,'Reviewer','name')
    
    review_about_restaurant = Relationship(review, "About", restaurant, 
                                           sentiment=row['SentimentClass'], 
                                           features=row['RestaurantFeature(spacy)'])
    graph.merge(review_about_restaurant)
    
    review_contains_food = Relationship(review, "Contains", food_node,
                                       sentiment=row['SentimentClass'],
                                       features=row['FoodFeature(spacy)'])
    graph.merge(review_contains_food)
    
    food_servedby_restaurant = Relationship(food_node, "Servedby", restaurant)
    graph.merge(food_servedby_restaurant)

In [56]:
output = pd.DataFrame(graph.run("MATCH (n) RETURN n"))#.to_table()

In [57]:
output.to_csv('./data/graph_output.csv')